In [2]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [3]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [4]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the weather from a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Search the weather data for a city"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [5]:
def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [13]:
from openai import OpenAI
client = OpenAI()

question = "What is the weather in Berlin?"

developer_prompt = """
You're a weather forecaster. 
You're given a city and your task is give the current temperature.
If you look up something in known weather data, return the data.
""".strip()

tools = [get_weather_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"city":"Berlin"}', call_id='call_fepuXrpOxgKMxB3NPeU3cLUY', name='get_weather', type='function_call', id='fc_6877e97867b881a2a7b1dbedc07e83430ef79db28aa19b61', status='completed')]

In [15]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [16]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the weather temperature for a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City for setting the weather"
            },
            "temp": {
                "type": "float",
                "description": "Temperature in celsius for a city"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [17]:
!pip install fastmcp

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 38.4 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11103 sha256=7a171afe028bde5d1564f78ad8dc5bc0b89674bcfd0ec706dfc0e36808832928
  Stored in directory: /home/codespace/.cache/pip/wheels/e0/e8/fc/8ab8aa326e33bc066ccd5f3ca9646eab4299881af933f94f09
Successfully built pyperclip
  Attempting uninstall: pydantic━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━  8/20 [starlette]
    Found existing installation: pydantic 2.11.5╺━━━━━━━━━━━━━━━━━━━━━━━  8/20 [starlette]
    Uninstalling pydantic-2.11.5:╺━━━━━━━━━━━━━━━━━━━━━━━  8/20 [starlette]
      Successfully uninstalled pydantic-2.11.57m╺━━━━━━━━━━━━━━━━━━━━━━━  8/20 [starlette]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20/20 [fastmcp]━ 18/20 

In [18]:
!fastmcp --version

2.10.5
